In [11]:
import pandas as pd

# Load the training data
train_df = pd.read_csv('D:/MLPR/timeseries/train.csv', parse_dates=['date'])

# Load the test data
test_df = pd.read_csv('D:/MLPR/timeseries/test.csv', parse_dates=['date'])

In [12]:
# One-hot encode the 'family' column
train_df = pd.get_dummies(train_df, columns=['family'])
test_df = pd.get_dummies(test_df, columns=['family'])

# Ensure both train and test data have the same columns after encoding
train_df, test_df = train_df.align(test_df, join='left', axis=1)


In [13]:
from sklearn.model_selection import train_test_split

# Define your features and target variable
X = train_df.drop(['sales', 'id', 'date'], axis=1)
y = train_df['sales']

# Split the data chronologically
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle=False)


In [17]:
# Replace special characters in column names with underscores
X_train.columns = X_train.columns.str.replace(r'\W', '_', regex=True)
X_valid.columns = X_valid.columns.str.replace(r'\W', '_', regex=True)

In [19]:
# Check the number of features
print(X_train.shape[1])  # Number of features in training set
print(len(X_train.columns))  # Number of feature names

35
35


In [20]:
# Ensure feature names are set
X_train.columns = [f'feature_{i}' for i in range(X_train.shape[1])]
X_valid.columns = [f'feature_{i}' for i in range(X_valid.shape[1])]

In [26]:
# Ensure non-negative predictions
y_pred = np.maximum(y_pred, 0)

In [28]:
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_log_error, mean_squared_error

# Initialize the model
model = LGBMRegressor()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_valid)

# Ensure non-negative predictions
y_pred = np.maximum(y_pred, 0)

# Evaluate the model using Mean Squared Logarithmic Error
rmsle = mean_squared_log_error(y_valid, y_pred, squared=False)
print(f'Root Mean Squared Logarithmic Error: {rmsle}')

# Alternatively, evaluate using Mean Squared Error
mse = mean_squared_error(y_valid, y_pred)
print(f'Mean Squared Error: {mse}')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004455 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 128
[LightGBM] [Info] Number of data points in the train set: 838860, number of used features: 35
[LightGBM] [Info] Start training from score 237.373651
Root Mean Squared Logarithmic Error: 2.014349266756285
Mean Squared Error: 156996.57431136616


In [32]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor

# Load the test data
test_df = pd.read_csv('D:/MLPR/timeseries/test.csv', parse_dates=['date'])

# One-hot encode the 'family' column
test_df = pd.get_dummies(test_df, columns=['family'])

# Load the training data and align columns
train_df = pd.read_csv('D:/MLPR/timeseries/train.csv', parse_dates=['date'])
train_df = pd.get_dummies(train_df, columns=['family'])
X_train = train_df.drop(['sales', 'id', 'date'], axis=1)

# Prepare test data with alignment
# Ensure test data has the same columns as the training data
test_df = test_df.drop(['id', 'date'], axis=1, errors='ignore')  # Drop columns if they exist

# Align columns
test_df = test_df.reindex(columns=X_train.columns, fill_value=0)

# Initialize the trained model (assuming it’s already trained)
# If the model is not trained, you need to train it here
# model = LGBMRegressor()
# model.fit(X_train, y_train)  # Only if you need to retrain, otherwise use the pre-trained model

# Make predictions
test_predictions = model.predict(test_df)

# Ensure non-negative predictions
test_predictions = np.maximum(test_predictions, 0)

# Prepare and save the submission DataFrame
submission_df = pd.DataFrame({
    'id': test_df.get('id', pd.Series([None]*len(test_df))),  # Handle case if 'id' is missing
    'sales': test_predictions
})

submission_df.to_csv('D:/MLPR/timeseries/submission.csv', index=False)

print("Predictions saved to 'D:/MLPR/timeseries/submission.csv'")

Predictions saved to 'D:/MLPR/timeseries/submission.csv'
